# Spatial Analysis of Zomato
***

## Introduction

## Business Understanding

## Data Understanding

In [46]:
# Importing relevant packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.geocoders import Nominatim
from warnings import filterwarnings
filterwarnings('ignore')

In [5]:
data = pd.read_csv(r'Data/zomato.csv')
type(data)

pandas.core.frame.DataFrame

above "data" belongs to dataframe data-structure

In [3]:
# Getting first 3 rows of data
data.head(3) 

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari


In [6]:
# Getting all the columns of dataframe
data.columns

Index(['url', 'address', 'name', 'online_order', 'book_table', 'rate', 'votes',
       'phone', 'location', 'rest_type', 'dish_liked', 'cuisines',
       'approx_cost(for two people)', 'reviews_list', 'menu_item',
       'listed_in(type)', 'listed_in(city)'],
      dtype='object')

In [7]:
# Getting dimensions of dataframe
data.shape

(51717, 17)

## Data Preprocessing

In [9]:
data.duplicated().sum()

0

In [10]:
data.isnull().sum()

url                                0
address                            0
name                               0
online_order                       0
book_table                         0
rate                            7775
votes                              0
phone                           1208
location                          21
rest_type                        227
dish_liked                     28078
cuisines                          45
approx_cost(for two people)      346
reviews_list                       0
menu_item                          0
listed_in(type)                    0
listed_in(city)                    0
dtype: int64

In [13]:
data.dropna(subset=['location'], inplace=True)
data.isnull().sum()

url                                0
address                            0
name                               0
online_order                       0
book_table                         0
rate                            7754
votes                              0
phone                           1187
location                           0
rest_type                        206
dish_liked                     28057
cuisines                          24
approx_cost(for two people)      325
reviews_list                       0
menu_item                          0
listed_in(type)                    0
listed_in(city)                    0
dtype: int64

In [15]:
df = data.copy()

In [16]:
df['location']

0                      Banashankari
1                      Banashankari
2                      Banashankari
3                      Banashankari
4                      Basavanagudi
                    ...            
51712                    Whitefield
51713                    Whitefield
51714                    Whitefield
51715    ITPL Main Road, Whitefield
51716    ITPL Main Road, Whitefield
Name: location, Length: 51696, dtype: object

In [19]:
df['location'] = df['location'] + ' , Bangalore , Karnataka , India'
df['location']

0             Banashankari , Bangalore , Karnataka , India
1             Banashankari , Bangalore , Karnataka , India
2             Banashankari , Bangalore , Karnataka , India
3             Banashankari , Bangalore , Karnataka , India
4             Basavanagudi , Bangalore , Karnataka , India
                               ...                        
51712           Whitefield , Bangalore , Karnataka , India
51713           Whitefield , Bangalore , Karnataka , India
51714           Whitefield , Bangalore , Karnataka , India
51715    ITPL Main Road, Whitefield , Bangalore , Karna...
51716    ITPL Main Road, Whitefield , Bangalore , Karna...
Name: location, Length: 51696, dtype: object

In [20]:
df.dtypes

url                            object
address                        object
name                           object
online_order                   object
book_table                     object
rate                           object
votes                           int64
phone                          object
location                       object
rest_type                      object
dish_liked                     object
cuisines                       object
approx_cost(for two people)    object
reviews_list                   object
menu_item                      object
listed_in(type)                object
listed_in(city)                object
dtype: object

# Extract Latitudes and Longitudes

In [23]:
# New dataframe
rest_loc = pd.DataFrame()

rest_loc['Name'] = df['location'].unique()
rest_loc

,Name
0,"Banashankari , Bangalore , Karnataka , India"
1,"Basavanagudi , Bangalore , Karnataka , India"
2,"Mysore Road , Bangalore , Karnataka , India"
3,"Jayanagar , Bangalore , Karnataka , India"
4,"Kumaraswamy Layout , Bangalore , Karnataka , I..."
...,...
88,"West Bangalore , Bangalore , Karnataka , India"
89,"Magadi Road , Bangalore , Karnataka , India"
90,"Yelahanka , Bangalore , Karnataka , India"
91,"Sahakara Nagar , Bangalore , Karnataka , India"


In [27]:
geolocator = Nominatim(user_agent='app1', timeout=None)

40.741059199999995

In [29]:
lat = []
lon = []

for name in rest_loc['Name']:
    location = geolocator.geocode(name)
    
    if location is None:
        lat.append(np.nan)
        lon.append(np.nan)
    else:
        lat.append(location.latitude)
        lon.append(location.longitude)
        
print(lat)

[12.9152186, 12.9417261, 12.9466619, 12.9292731, 12.9081487, 12.9274413, 12.9658625, 12.9055682, 12.9120761, 12.9287596, 12.965717999999999, 13.00196975, 12.9153309, 12.911275849999999, 12.9124938, 12.9089453, 12.9854892, 12.848759900000001, 12.9116225, 12.9552572, 12.920441, 12.9489339, 12.9575547, 12.9348429, 12.9408685, 12.9668257, 12.9364846, 13.0464531, 12.9327778, 12.93103185, 12.9696365, 12.98970315, 12.9572222, 12.9732913, 12.9277245, 12.9986827, 13.0227204, 12.9755264, 12.9750849, 12.9683054, 13.0328037, 12.9778793, 12.9660577, 12.986391, 12.9802639, 12.9744255, 12.987043, 12.9844724, 12.9822323, 12.988721250000001, 13.0358698, 12.9624669, 12.945245, 12.9678074, 13.0076091, 13.0027353, 12.9931876, 13.0093455, 12.9390255, 12.9756601, 12.957998, 12.9730367, 12.9578658, 12.957434549999999, 12.9864828, 12.944569, 13.007516, 12.9243692, 12.9282918, 12.9340114, 12.9211258, 12.9882338, 13.0141618, 13.022234699999998, 13.0422794, 13.0258087, 13.0221416, 13.0421236, 13.0784743, nan, 12

In [32]:
rest_loc['lat'] = lat
rest_loc['lon'] = lon

rest_loc

,Name,lat,lon
0,"Banashankari , Bangalore , Karnataka , India",12.915219,77.573621
1,"Basavanagudi , Bangalore , Karnataka , India",12.941726,77.575502
2,"Mysore Road , Bangalore , Karnataka , India",12.946662,77.530090
3,"Jayanagar , Bangalore , Karnataka , India",12.929273,77.582423
4,"Kumaraswamy Layout , Bangalore , Karnataka , I...",12.908149,77.555318
...,...,...,...
88,"West Bangalore , Bangalore , Karnataka , India",13.022235,77.567183
89,"Magadi Road , Bangalore , Karnataka , India",12.975608,77.555356
90,"Yelahanka , Bangalore , Karnataka , India",13.100698,77.596345
91,"Sahakara Nagar , Bangalore , Karnataka , India",13.062147,77.580061


In [33]:
rest_loc.isnull().sum()

Name    0
lat     2
lon     2
dtype: int64

In [36]:
rest_loc[rest_loc['lat'].isnull()]

,Name,lat,lon
79,"Rammurthy Nagar , Bangalore , Karnataka , India",NaN,NaN
85,"Sadashiv Nagar , Bangalore , Karnataka , India",NaN,NaN


Google search of coordinates for missing

In [48]:
# for Rammurthy Nagar , Bangalore 13.0163° N, 77.6785° E
rest_loc['lat'][79] = 13.0184435
rest_loc['lon'][79] = 77.6781215

rest_loc['lat'][85] = 13.010316
rest_loc['lon'][85] = 77.580569

rest_loc.isnull().sum()

Name    0
lat     0
lon     0
dtype: int64

## Restaurants Density

77.6781215

13.010316